In [6]:
import pandas as pd

# Cargar el archivo CSV
Ames = pd.read_csv('Ames.csv')

# Identificar las 10 casas más caras basadas en SalePrice con características clave
top_10_expensive_homes = Ames.nlargest(10, 'SalePrice')

# Seleccionar características clave
features = ['SalePrice', 'GrLivArea', 'OverallQual', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces']
top_10_df = top_10_expensive_homes[features]

# Mostrar el DataFrame
print(top_10_df)

      SalePrice  GrLivArea  OverallQual KitchenQual  TotRmsAbvGrd  Fireplaces
1007     755000       4316           10          Ex            10           2
1325     625000       3627           10          Gd            10           1
65       615000       2470           10          Ex             7           2
584      611657       2364            9          Ex            11           2
1639     610000       2674           10          Ex             8           2
615      591587       2338            9          Gd             8           2
2087     584500       3500            9          Ex            11           1
1112     555000       2402           10          Ex            10           2
802      538000       3279            8          Ex            12           1
407      535000       2698           10          Ex            11           1


In [8]:
import pandas as pd

# Cargar los datos
Ames = pd.read_csv('Ames.csv')

# Obtener las 10 casas más caras según SalePrice
top_10_expensive_homes = Ames.nlargest(10, 'SalePrice')

# Seleccionar las características clave
features = ['SalePrice', 'GrLivArea', 'OverallQual', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces']
top_10_df = top_10_expensive_homes[features]

# Refinar la búsqueda: calidad general 10, cocina excelente y al menos 2 chimeneas
elite = top_10_df.query('OverallQual == 10 & KitchenQual == "Ex" & Fireplaces >= 2')

# Mostrar resultados
print(elite)


      SalePrice  GrLivArea  OverallQual KitchenQual  TotRmsAbvGrd  Fireplaces
1007     755000       4316           10          Ex            10           2
65       615000       2470           10          Ex             7           2
1639     610000       2674           10          Ex             8           2
1112     555000       2402           10          Ex            10           2


In [10]:
import pandas as pd

# Cargar los datos
Ames = pd.read_csv('Ames.csv')

# Obtener las 10 casas más caras según SalePrice
top_10_expensive_homes = Ames.nlargest(10, 'SalePrice')

# Seleccionar las características clave
features = ['SalePrice', 'GrLivArea', 'OverallQual', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces']
top_10_df = top_10_expensive_homes[features]

# Filtrar casas con alta calidad, cocina excelente y al menos 2 chimeneas
elite = top_10_df.query('OverallQual == 10 & KitchenQual == "Ex" & Fireplaces >= 2').copy()

# Calcular el precio por pie cuadrado (Price per Square Foot)
elite['PSF'] = elite['SalePrice'] / elite['GrLivArea']

# Mostrar casas ordenadas por PSF
print(elite.sort_values(by='PSF'))

      SalePrice  GrLivArea  OverallQual KitchenQual  TotRmsAbvGrd  Fireplaces  \
1007     755000       4316           10          Ex            10           2   
1639     610000       2674           10          Ex             8           2   
1112     555000       2402           10          Ex            10           2   
65       615000       2470           10          Ex             7           2   

             PSF  
1007  174.930491  
1639  228.122663  
1112  231.057452  
65    248.987854  


In [13]:
import pandas as pd

Ames = pd.read_csv('Ames.csv')

# Cross check entire homes to search for better value
Ames['PSF'] = Ames['SalePrice']/Ames['GrLivArea']
value = Ames.query('PSF < 175 & OverallQual == 10 & KitchenQual == "Ex" & Fireplaces >=2')
print(value[['SalePrice', 'GrLivArea', 'OverallQual', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 'PSF']])

      SalePrice  GrLivArea  OverallQual KitchenQual  TotRmsAbvGrd  Fireplaces  \
1007     755000       4316           10          Ex            10           2   
2003     475000       3608           10          Ex            12           2   

             PSF  
1007  174.930491  
2003  131.651885  


In [15]:
import pandas as pd
import folium

# Cargar los datos
Ames = pd.read_csv('Ames.csv')

# Calcular el precio por pie cuadrado (PSF)
Ames['PSF'] = Ames['SalePrice'] / Ames['GrLivArea']

# Filtrar propiedades con alta calidad y buena relación calidad-precio
value = Ames.query('PSF < 175 & OverallQual == 10 & KitchenQual == "Ex" & Fireplaces >= 2')

# Obtener los índices de las observaciones seleccionadas
final_observation_indexes = value.index.tolist()

# Filtrar dataset para obtener latitud y longitud
final_locations = Ames.loc[final_observation_indexes, ['Latitude', 'Longitude']]

# Crear un mapa centrado en el promedio de las ubicaciones finales
map_center = [final_locations['Latitude'].mean(), final_locations['Longitude'].mean()]
value_map = folium.Map(location=map_center, zoom_start=12)

# Agregar marcadores con información adicional
for idx, row in final_locations.iterrows():
    info = value.loc[idx, ['SalePrice', 'GrLivArea', 'OverallQual', 'KitchenQual',
                           'TotRmsAbvGrd', 'Fireplaces', 'PSF']]
    popup_text = f"""
    <b>Index:</b> {idx}<br>
    <b>SalePrice:</b> ${info['SalePrice']}<br>
    <b>GrLivArea:</b> {info['GrLivArea']} sqft<br>
    <b>OverallQual:</b> {info['OverallQual']}<br>
    <b>KitchenQual:</b> {info['KitchenQual']}<br>
    <b>TotRmsAbvGrd:</b> {info['TotRmsAbvGrd']}<br>
    <b>Fireplaces:</b> {info['Fireplaces']}<br>
    <b>PSF:</b> ${info['PSF']:.2f} /sqft
    """
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(value_map)

# Guardar el mapa en un archivo HTML
value_map.save('value_map.html')

In [17]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Cargar los datos
Ames = pd.read_csv('Ames.csv')

# Filtrar filas sin coordenadas geográficas
Ames_Heat = Ames.dropna(subset=['Latitude', 'Longitude'])

# Agrupar por vecindario y calcular la latitud y longitud promedio
neighborhood_locs = Ames_Heat.groupby('Neighborhood') \
    .agg({'Latitude': 'mean', 'Longitude': 'mean'}) \
    .reset_index()

# Centro del mapa en Ames, Iowa
ames_map_center = [Ames_Heat['Latitude'].mean(), Ames_Heat['Longitude'].mean()]
ames_heatmap = folium.Map(location=ames_map_center, zoom_start=12)

# Extraer datos para el HeatMap
heat_data = [(lat, lon) for lat, lon in zip(Ames_Heat['Latitude'], Ames_Heat['Longitude'])]

# Crear y añadir la capa HeatMap
HeatMap(heat_data, radius=12).add_to(ames_heatmap)

# Añadir una bandera negra por vecindario
for index, row in neighborhood_locs.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Neighborhood'],
        icon=folium.Icon(color='black', icon='flag')
    ).add_to(ames_heatmap)

# Guardar el mapa en un archivo HTML
ames_heatmap.save('ames_heatmap.html')